## Monitoramento de Clima - Metodologia ClimaTempo

`API ClimaTempo`

**Resumo**: Captura dos dados meteorológicos via API da ClimaTempo <br>
**Fonte**: http://apiadvisor.climatempo.com.br/doc/index.html#api-Forecast-Forecast15DaysByCity

In [1]:
# Libraries
import pandas as pd
import numpy as np
import time
import sys
import csv
import json
import requests
import re
import warnings
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from unidecode import unidecode

In [2]:
# Configuração básica de layout/exibição
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_colwidth = 100
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

# Reset dos parâmetros
sys.argv = [sys.argv[0]]

# Acrescentar outras formas de coleta, exemplo abaixo
# sys.argv.append("DAY")

In [3]:
# Login Parameters
LOGIN = {
    'token': '3134eb78c753bd4079786bf4865850a4',
}

In [4]:
# Setting the URL template
cities_url = "http://apiadvisor.climatempo.com.br/api/v1/locale/city?country=BR&token={token}"
current_weather = "http://apiadvisor.climatempo.com.br/api/v1/weather/locale/{city}/current?token={token}"
forecast_72h = "http://apiadvisor.climatempo.com.br/api/v1/forecast/locale/{city}/hours/72?token={token}"
forecast_15d = "http://apiadvisor.climatempo.com.br/api/v1/forecast/locale/{city}/days/15?token={token}"
forecast_15d_v2 = "http://apiadvisor.climatempo.com.br/api/v2/forecast/locale/{city}/days/15?token={token}"
forecast_3d_region = "http://apiadvisor.climatempo.com.br/api/v1/forecast/region/{region}?token={token}"

In [5]:
# Setting the city of São Paulo
input_url = 'http://apiadvisor.climatempo.com.br/api-manager/user-token/{token}/locales'
payload = 'localeId[]=3477'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

# Putting the new city to be tracked
response = requests.request('PUT', input_url.format(token = LOGIN['token']), headers = headers, data = payload)

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data)
data

,error,detail
0,True,Your free plan only allows to update cities after 24 hours from the last update (2022-12-08 12:5...


In [6]:
# Validating the defined city
response = requests.request('GET', 'http://apiadvisor.climatempo.com.br/api-manager/user-token/01abb86c1e0c2434c4d72c32a4dd70ad/locales')

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data)
data

,max_allowed,locales
0,1,[3477]


In [7]:
# Search country's cities list
response = requests.request('GET', cities_url.format(token = LOGIN['token']))

In [8]:
# Getting the corresponding output
cities = json.loads(response.text)
cities = pd.json_normalize(cities)
cities

,id,name,state,country
0,8285,Abadia de Goiás,GO,BR
1,8529,Abadia dos Dourados,MG,BR
2,8286,Abadiânia,GO,BR
3,6868,Abaeté,MG,BR
4,7696,Abaetetuba,PA,BR
...,...,...,...,...
6113,7995,Xique-Xique,BA,BR
6114,7248,Zabelê,PB,BR
6115,3562,Zacarias,SP,BR
6116,4496,Zé Doca,MA,BR


In [10]:
# Sampling of the major city (São Paulo)
cities.loc[cities['name'] == 'São Paulo']

,id,name,state,country
5297,3477,São Paulo,SP,BR


In [11]:
## CURRENT WEATHER
response = requests.request('GET', current_weather.format(city = '3477', token = LOGIN['token']))

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data)
data

,id,name,state,country,data.temperature,data.wind_direction,data.wind_velocity,data.humidity,data.condition,data.pressure,data.icon,data.sensation,data.date
0,3477,São Paulo,SP,BR,7,NW,39,70,Muitas nuvens,1015,2,None,2022-05-18 09:38:26


In [12]:
## FORECAST 72 HOURS
response = requests.request('GET', forecast_72h.format(city = '3477', token = LOGIN['token']))

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data, record_path = ['data'], meta = ['name', 'state', 'country'])
data = data[['name', 'state', 'country', 'date', 'date_br', 'temperature.temperature', 'humidity.humidity', 'pressure.pressure',
       'rain.precipitation', 'wind.velocity', 'wind.direction',
       'wind.directiondegrees', 'wind.gust']]
data = data.rename(columns = {
    'humidity.humidity': 'humidity',
    'pressure.pressure': 'pressure',
    'rain.precipitation': 'rain_precipitation',
    'wind.velocity': 'wind_velocity',
    'wind.direction': 'wind_direction',
    'wind.directiondegrees': 'wind_directiondegrees',
    'wind.gust': 'wind_gust',
    'temperature.temperature': 'temperature'
})
data

,name,state,country,date,date_br,temperature,humidity,pressure,rain_precipitation,wind_velocity,wind_direction,wind_directiondegrees,wind_gust
0,São Paulo,SP,BR,2022-05-18 00:00:00,18/05/2022 00:00:00,14,50.10,921.20,0,27.60,WNW,292.70,47.80
1,São Paulo,SP,BR,2022-05-18 01:00:00,18/05/2022 01:00:00,13,53.10,921.10,0,29.00,W,287.80,51.90
2,São Paulo,SP,BR,2022-05-18 02:00:00,18/05/2022 02:00:00,12,55.60,922.70,0,29.40,W,284.00,53.60
3,São Paulo,SP,BR,2022-05-18 03:00:00,18/05/2022 03:00:00,11,57.80,922.40,0,29.00,W,284.00,51.40
4,São Paulo,SP,BR,2022-05-18 04:00:00,18/05/2022 04:00:00,10,59.60,921.40,0,28.60,W,287.40,50.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,São Paulo,SP,BR,2022-05-20 19:00:00,20/05/2022 19:00:00,17,69.10,923.80,0,12.50,SSE,175.50,14.10
68,São Paulo,SP,BR,2022-05-20 20:00:00,20/05/2022 20:00:00,16,73.00,923.80,0,13.90,SSE,174.80,17.60
69,São Paulo,SP,BR,2022-05-20 21:00:00,20/05/2022 21:00:00,14,73.90,923.90,0,14.30,SSE,175.40,20.70
70,São Paulo,SP,BR,2022-05-20 22:00:00,20/05/2022 22:00:00,13,75.90,924.90,0,14.50,SSE,174.80,21.90


In [13]:
# GBQ schema information
schema_main = [
    {'name': 'name', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY NAME'},
    {'name': 'state', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY STATE'},
    {'name': 'country', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY COUNTRY'},
    {'name': 'date', 'type': 'DATETIME', 'mode': 'NULLABLE', 'description': 'DATE IN FORMAT "YYYY-MM-DD" (EX: 2017-10-01)'},
    {'name': 'date_br', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DATE IN BRAZILIAN FORMAT "DD/MM/YYYY" (EX: 01/10/2017)'},
    {'name': 'temperature', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'humidity', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY (%)'},
    {'name': 'pressure', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'PRESSURE (HPA)'},
    {'name': 'rain_precipitation', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'PRECIPITATION IN MILLIMETER'},
    {'name': 'wind_velocity', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND INTENSITY IN KM/H'},
    {'name': 'wind_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION'},
    {'name': 'wind_directiondegrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION IN DEGREE'},
    {'name': 'wind_gust', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND GUST IN KM/H'},
]

In [14]:
## FORECAST 15 DAYS V1
response = requests.request('GET', forecast_15d.format(city = '3477', token = LOGIN['token']))

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data, record_path = ['data'], meta = ['id','name', 'state', 'country'])
data = data.rename(columns = {
                    'date': 'date',
                    'date_br': 'date_br',
                    'humidity.min': 'humidity_min',
                    'humidity.max': 'humidity_max',
                    'humidity.dawn.min': 'humidity_dawn_min',
                    'humidity.dawn.max': 'humidity_dawn_max',
                    'humidity.morning.min': 'humidity_morning_min',
                    'humidity.morning.max': 'humidity_morning_max',
                    'humidity.afternoon.min': 'humidity_afternoon_min',
                    'humidity.afternoon.max': 'humidity_afternoon_max',
                    'humidity.night.min': 'humidity_night_min',
                    'humidity.night.max': 'humidity_night_max',
                    'pressure.pressure': 'pressure_pressure',
                    'rain.probability': 'rain_probability',
                    'rain.precipitation': 'rain_precipitation',
                    'wind.velocity_min': 'wind_velocity_min',
                    'wind.velocity_max': 'wind_velocity_max',
                    'wind.velocity_avg': 'wind_velocity_avg',
                    'wind.gust_max': 'wind_gust_max',
                    'wind.direction_degrees': 'wind_direction_degrees',
                    'wind.direction': 'wind_direction',
                    'wind.dawn.direction': 'wind_dawn_direction',
                    'wind.dawn.direction_degrees': 'wind_dawn_direction_degrees',
                    'wind.dawn.gust_max': 'wind_dawn_gust_max',
                    'wind.dawn.velocity_max': 'wind_dawn_velocity_max',
                    'wind.dawn.velocity_avg': 'wind_dawn_velocity_avg',
                    'wind.morning.direction': 'wind_morning_direction',
                    'wind.morning.direction_degrees': 'wind_morning_direction_degrees',
                    'wind.morning.gust_max': 'wind_morning_gust_max',
                    'wind.morning.velocity_max': 'wind_morning_velocity_max',
                    'wind.morning.velocity_avg': 'wind_morning_velocity_avg',
                    'wind.afternoon.direction': 'wind_afternoon_direction',
                    'wind.afternoon.direction_degrees': 'wind_afternoon_direction_degrees',
                    'wind.afternoon.gust_max': 'wind_afternoon_gust_max',
                    'wind.afternoon.velocity_max': 'wind_afternoon_velocity_max',
                    'wind.afternoon.velocity_avg': 'wind_afternoon_velocity_avg',
                    'wind.night.direction': 'wind_night_direction',
                    'wind.night.direction_degrees': 'wind_night_direction_degrees',
                    'wind.night.gust_max': 'wind_night_gust_max',
                    'wind.night.velocity_max': 'wind_night_velocity_max',
                    'wind.night.velocity_avg': 'wind_night_velocity_avg',
                    'uv.max': 'uv_max',
                    'thermal_sensation.min': 'thermal_sensation_min',
                    'thermal_sensation.max': 'thermal_sensation_max',
                    'text_icon.icon.dawn': 'text_icon_icon_dawn',
                    'text_icon.icon.morning': 'text_icon_icon_morning',
                    'text_icon.icon.afternoon': 'text_icon_icon_afternoon',
                    'text_icon.icon.night': 'text_icon_icon_night',
                    'text_icon.icon.day': 'text_icon_icon_day',
                    'text_icon.text.pt': 'text_icon_text_pt',
                    'text_icon.text.en': 'text_icon_text_en',
                    'text_icon.text.es': 'text_icon_text_es',
                    'text_icon.text.phrase.reduced': 'text_icon_text_phrase_reduced',
                    'text_icon.text.phrase.morning': 'text_icon_text_phrase_morning',
                    'text_icon.text.phrase.afternoon': 'text_icon_text_phrase_afternoon',
                    'text_icon.text.phrase.night': 'text_icon_text_phrase_night',
                    'text_icon.text.phrase.dawn': 'text_icon_text_phrase_dawn',
                    'temperature.min': 'temperature_min',
                    'temperature.max': 'temperature_max',
                    'temperature.dawn.min': 'temperature_dawn_min',
                    'temperature.dawn.max': 'temperature_dawn_max',
                    'temperature.morning.min': 'temperature_morning_min',
                    'temperature.morning.max': 'temperature_morning_max',
                    'temperature.afternoon.min': 'temperature_afternoon_min',
                    'temperature.afternoon.max': 'temperature_afternoon_max',
                    'temperature.night.min': 'temperature_night_min',
                    'temperature.night.max': 'temperature_night_max',
                    'cloud_coverage.low': 'cloud_coverage_low',
                    'cloud_coverage.mid': 'cloud_coverage_mid',
                    'cloud_coverage.high': 'cloud_coverage_high',
                    'cloud_coverage.dawn.low': 'cloud_coverage_dawn_low',
                    'cloud_coverage.dawn.mid': 'cloud_coverage_dawn_mid',
                    'cloud_coverage.dawn.high': 'cloud_coverage_dawn_high',
                    'cloud_coverage.morning.low': 'cloud_coverage_morning_low',
                    'cloud_coverage.morning.mid': 'cloud_coverage_morning_mid',
                    'cloud_coverage.morning.high': 'cloud_coverage_morning_high',
                    'cloud_coverage.afternoon.low': 'cloud_coverage_afternoon_low',
                    'cloud_coverage.afternoon.mid': 'cloud_coverage_afternoon_mid',
                    'cloud_coverage.afternoon.high': 'cloud_coverage_afternoon_high',
                    'cloud_coverage.night.low': 'cloud_coverage_night_low',
                    'cloud_coverage.night.mid': 'cloud_coverage_night_mid',
                    'cloud_coverage.night.high': 'cloud_coverage_night_high',
                    'sun.sunrise': 'sun_sunrise',
                    'sun.sunset': 'sun_sunset',
                    'id': 'id',
                    'name': 'name',
                    'state': 'state',
                    'country': 'country'
})
data = data[['date', 'date_br', 'id', 'name', 'state', 'country', 'humidity_min', 'humidity_max', 'humidity_dawn_min',
 'humidity_dawn_max', 'humidity_morning_min', 'humidity_morning_max', 'humidity_afternoon_min', 'humidity_afternoon_max',
 'humidity_night_min', 'humidity_night_max', 'pressure_pressure', 'rain_probability', 'rain_precipitation',
 'wind_velocity_min', 'wind_velocity_max', 'wind_velocity_avg', 'wind_gust_max', 'wind_direction_degrees', 'wind_direction',
 'wind_dawn_direction', 'wind_dawn_direction_degrees', 'wind_dawn_gust_max', 'wind_dawn_velocity_max', 'wind_dawn_velocity_avg',
 'wind_morning_direction', 'wind_morning_direction_degrees', 'wind_morning_gust_max', 'wind_morning_velocity_max',
 'wind_morning_velocity_avg', 'wind_afternoon_direction', 'wind_afternoon_direction_degrees', 'wind_afternoon_gust_max',
 'wind_afternoon_velocity_max', 'wind_afternoon_velocity_avg', 'wind_night_direction', 'wind_night_direction_degrees',
 'wind_night_gust_max', 'wind_night_velocity_max', 'wind_night_velocity_avg', 'uv_max', 'thermal_sensation_min',
 'thermal_sensation_max', 'text_icon_icon_dawn', 'text_icon_icon_morning', 'text_icon_icon_afternoon', 'text_icon_icon_night',
 'text_icon_icon_day', 'text_icon_text_pt', 'text_icon_text_en', 'text_icon_text_es', 'text_icon_text_phrase_reduced', 'text_icon_text_phrase_morning',
 'text_icon_text_phrase_afternoon', 'text_icon_text_phrase_night', 'text_icon_text_phrase_dawn', 'temperature_min',
 'temperature_max', 'temperature_dawn_min', 'temperature_dawn_max', 'temperature_morning_min', 'temperature_morning_max',
 'temperature_afternoon_min', 'temperature_afternoon_max', 'temperature_night_min', 'temperature_night_max', 'cloud_coverage_low',
 'cloud_coverage_mid', 'cloud_coverage_high', 'cloud_coverage_dawn_low', 'cloud_coverage_dawn_mid', 'cloud_coverage_dawn_high',
 'cloud_coverage_morning_low', 'cloud_coverage_morning_mid', 'cloud_coverage_morning_high', 'cloud_coverage_afternoon_low',
 'cloud_coverage_afternoon_mid', 'cloud_coverage_afternoon_high', 'cloud_coverage_night_low', 'cloud_coverage_night_mid',
 'cloud_coverage_night_high', 'sun_sunrise', 'sun_sunset']]
data

,date,date_br,id,name,state,country,humidity_min,humidity_max,humidity_dawn_min,humidity_dawn_max,humidity_morning_min,humidity_morning_max,humidity_afternoon_min,humidity_afternoon_max,humidity_night_min,humidity_night_max,pressure_pressure,rain_probability,rain_precipitation,wind_velocity_min,wind_velocity_max,wind_velocity_avg,wind_gust_max,wind_direction_degrees,wind_direction,wind_dawn_direction,wind_dawn_direction_degrees,wind_dawn_gust_max,wind_dawn_velocity_max,wind_dawn_velocity_avg,wind_morning_direction,wind_morning_direction_degrees,wind_morning_gust_max,wind_morning_velocity_max,wind_morning_velocity_avg,wind_afternoon_direction,wind_afternoon_direction_degrees,wind_afternoon_gust_max,wind_afternoon_velocity_max,wind_afternoon_velocity_avg,wind_night_direction,wind_night_direction_degrees,wind_night_gust_max,wind_night_velocity_max,wind_night_velocity_avg,uv_max,thermal_sensation_min,thermal_sensation_max,text_icon_icon_dawn,text_icon_icon_morning,text_icon_icon_afternoon,text_icon_icon_night,text_icon_icon_day,text_icon_text_pt,text_icon_text_en,text_icon_text_es,text_icon_text_phrase_reduced,text_icon_text_phrase_morning,text_icon_text_phrase_afternoon,text_icon_text_phrase_night,text_icon_text_phrase_dawn,temperature_min,temperature_max,temperature_dawn_min,temperature_dawn_max,temperature_morning_min,temperature_morning_max,temperature_afternoon_min,temperature_afternoon_max,temperature_night_min,temperature_night_max,cloud_coverage_low,cloud_coverage_mid,cloud_coverage_high,cloud_coverage_dawn_low,cloud_coverage_dawn_mid,cloud_coverage_dawn_high,cloud_coverage_morning_low,cloud_coverage_morning_mid,cloud_coverage_morning_high,cloud_coverage_afternoon_low,cloud_coverage_afternoon_mid,cloud_coverage_afternoon_high,cloud_coverage_night_low,cloud_coverage_night_mid,cloud_coverage_night_high,sun_sunrise,sun_sunset
0,2022-05-18,18/05/2022,3477,São Paulo,SP,BR,31,58,45,55,40,58,35,41,50,67,919.40,0,0,22,34,28,57,283,W,W,271,47,27,25,W,284,45,28,25,W,291,48,34,31,W,283,57,33,33,6.70,2,15,2n,2,2,2n,2,Sol com algumas nuvens,Mostly sunny.,Sol con algunas nubes. No llueve.,Sol com algumas nuvens. Não chove.,Sol com algumas nuvens,Sol com algumas nuvens,Algumas nuvens,Algumas nuvens,6,15,9,13,6,8,9,15,11,15,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,0.00,0,0,06:34:00,17:31:00
1,2022-05-19,19/05/2022,3477,São Paulo,SP,BR,44,76,69,75,52,76,44,49,52,63,919.00,0,0,15,30,20,53,254,WSW,WSW,267,53,30,27,WSW,258,40,22,21,SW,243,28,18,16,SW,237,25,18,17,6.50,3,17,2rn,2r,2,2n,2,Sol com algumas nuvens,Mostly sunny.,Sol con algunas nubes. No llueve.,Dia de sol com algumas nuvens e névoa ao amanhecer. Noite com poucas nuvens.,Sol com muitas nuvens,Sol com algumas nuvens,Algumas nuvens,Muitas nuvens,7,17,7,9,7,8,8,17,14,17,16.60,0,0,0.00,0,0,1.30,0,0,53.90,0,0,11.00,0,0,06:35:00,17:30:00
2,2022-05-20,20/05/2022,3477,São Paulo,SP,BR,48,79,67,78,54,79,48,56,63,79,923.90,0,0,12,16,13,27,188,S,SSW,209,27,14,13,S,189,25,16,14,SSE,179,23,16,14,SSE,174,22,14,14,6.70,4,18,2rn,2r,2,2n,2,Sol com algumas nuvens,Mostly sunny.,Sol con algunas nubes. No llueve.,Dia de sol com algumas nuvens e névoa ao amanhecer. Noite com poucas nuvens.,Sol com muitas nuvens,Sol com algumas nuvens,Algumas nuvens,Muitas nuvens,8,18,10,12,8,9,11,18,12,18,11.80,0,0,0.50,0,0,0.00,0,0,30.40,0,0,16.50,0,0,06:35:00,17:30:00
3,2022-05-21,21/05/2022,3477,São Paulo,SP,BR,54,88,80,87,59,88,54,58,65,80,928.50,90,3,9,15,12,22,147,SE,SSE,170,21,12,11,SSE,159,22,14,11,ESE,136,22,15,14,ESE,130,21,14,13,5.30,6,19,2rn,2r,4,2n,4,Sol e Chuva,Sun and rain.,Sol y aumento de nubes. Chubascos aislados en la tarde y en la noche.,Sol com muitas nuvens de manhã e pancadas de chuva à tarde. À noite a chuva para.,Sol com muitas nuvens,Sol e Chuva,Algumas nuvens,Muitas nuvens,10,19,10,12,10,12,13,19,15,19,12.20,0,0,0.00,0,0,0.00,0,0,44.90,0,0,3.70,0,0,06:35:00,17:30:00
4,2022-05-22,22/05/2022,3477,São Paulo,SP,BR,35,86,82,86,52,85,35,43,49,63,927.90,0,0,2,12,9,20,82,ENE,ESE,115,20,11,11,ENE,74,20,12,11

In [15]:
# GBQ schema information
schema_main = [
    {'name': 'date', 'type': 'DATETIME', 'mode': 'NULLABLE', 'description': 'DATE IN FORMAT "YYYY-MM-DD" (EX: 2017-10-01)'},
    {'name': 'date_br', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DATE IN BRAZILIAN FORMAT "DD/MM/YYYY" (EX: 01/10/2017)'},
    {'name': 'id', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY ID'},
    {'name': 'name', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY NAME'},
    {'name': 'state', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY STATE'},
    {'name': 'country', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY COUNTRY'},
    {'name': 'humidity_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN (%)'},
    {'name': 'humidity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX (%)'},
    {'name': 'humidity_dawn_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN DAWN (%)'},
    {'name': 'humidity_dawn_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX DAWN (%)'},
    {'name': 'humidity_morning_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN MORNING (%)'},
    {'name': 'humidity_morning_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX MORNING (%)'},
    {'name': 'humidity_afternoon_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN EVENING (%)'},
    {'name': 'humidity_afternoon_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX EVENING (%)'},
    {'name': 'humidity_night_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN NIGTH (%)'},
    {'name': 'humidity_night_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX NIGTH (%)'},
    {'name': 'pressure_pressure', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'PRESSURE (HPA)'},
    {'name': 'rain_probability', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RAIN PROBABILITY IN PERCENT'},
    {'name': 'rain_precipitation', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'PRECIPITATION IN MILIMETER'},
    {'name': 'wind_velocity_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM WIND INTENSITY IN KM/H'},
    {'name': 'wind_velocity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM WIND INTENSITY IN KM/H'},
    {'name': 'wind_velocity_avg', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'AVERAGE WIND INTENSITY IN KM/H'},
    {'name': 'wind_gust_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM WIND GUST IN KM/H'},
    {'name': 'wind_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION IN DEGREES'},
    {'name': 'wind_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION'},
    {'name': 'wind_dawn_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DAWN WIND DIRECTION'},
    {'name': 'wind_dawn_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN WIND DIRECTION IN DEGREES'},
    {'name': 'wind_dawn_gust_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN WIND MAXIMUM GUST IN KM/H'},
    {'name': 'wind_dawn_velocity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN WIND MAXIMUM INTENSITY IN KM/H'},
    {'name': 'wind_dawn_velocity_avg', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN AVERAGE WIND INTENSITY IN KM/H'},
    {'name': 'wind_morning_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'MORNING WIND DIRECTION'},
    {'name': 'wind_morning_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING WIND DIRECTION IN DEGREES'},
    {'name': 'wind_morning_gust_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING WIND MAXIMUM GUST IN KM/H'},
    {'name': 'wind_morning_velocity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING WIND MAXIMUM INTENSITY IN KM/H'},
    {'name': 'wind_morning_velocity_avg', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING AVERAGE WIND INTENSITY IN KM/H'},
    {'name': 'wind_afternoon_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'EVENING WIND DIRECTION'},
    {'name': 'wind_afternoon_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING WIND DIRECTION IN DEGREES'},
    {'name': 'wind_afternoon_gust_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING WIND MAXIMUM GUST IN KM/H'},
    {'name': 'wind_afternoon_velocity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING WIND MAXIMUM INTENSITY IN KM/H'},
    {'name': 'wind_afternoon_velocity_avg', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING AVERAGE WIND INTENSITY IN KM/H'},
    {'name': 'wind_night_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'NIGTH WIND DIRECTION'},
    {'name': 'wind_night_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH WIND DIRECTION IN DEGRESS'},
    {'name': 'wind_night_gust_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH WIND MAXIMUM GUST IN KM/H'},
    {'name': 'wind_night_velocity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH WIND MAXIMUM INTENSITY IN KM/H'},
    {'name': 'wind_night_velocity_avg', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH AVERAGE WIND INTENSITY IN KM/H'},
    {'name': 'uv_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'UV MAXIMUN INDEX'},
    {'name': 'thermal_sensation_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM THERMAL SENSATION(°C)'},
    {'name': 'thermal_sensation_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM THERMAL SENSATION(°C)'},
    {'name': 'text_icon_icon_dawn', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DAWN ICON'},
    {'name': 'text_icon_icon_morning', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'MORNING ICON'},
    {'name': 'text_icon_icon_afternoon', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'EVENING ICON'},
    {'name': 'text_icon_icon_night', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'NIGTH ICON'},
    {'name': 'text_icon_icon_day', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DAY ICON'},
    {'name': 'text_icon_text_pt', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'FORECAST TEXT IN PORTUGUESE'},
    {'name': 'text_icon_text_en', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'FORECAST TEXT IN ENGLISH'},
    {'name': 'text_icon_text_es', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'FORECAST TEXT IN SPANISH'},
    {'name': 'text_icon_text_phrase_reduced', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'REDUCED FORECAST PHRASE DAY'},
    {'name': 'text_icon_text_phrase_morning', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'MORNING FORECAST PHRASE'},
    {'name': 'text_icon_text_phrase_afternoon', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'EVENING FORECAST PHRASE'},
    {'name': 'text_icon_text_phrase_night', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'NIGTH FORECAST PHRASE'},
    {'name': 'text_icon_text_phrase_dawn', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DAWN FORECAST PHRASE'},
    {'name': 'temperature_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM TEMPERATURE IN CELSIUS DERGREE'},
    {'name': 'temperature_dawn_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_dawn_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN MAXIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_morning_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_morning_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_afternoon_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_afternoon_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'EVENING MAXIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_night_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_night_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGTH MAXIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'cloud_coverage_low', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'LOW-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_mid', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MID-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_high', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'HIGH-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_dawn_low', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN LOW-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_dawn_mid', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN MID-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_dawn_high', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'DAWN HIGH-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_morning_low', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING LOW-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_morning_mid', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING MID-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_morning_high', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MORNING HIGH-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_afternoon_low', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'AFTERNOON LOW-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_afternoon_mid', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'AFTERNOON MID-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_afternoon_high', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'AFTERNOON HIGH-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_night_low', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGHT LOW-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_night_mid', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGHT MID-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'cloud_coverage_night_high', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'NIGHT HIGH-LEVEL CLOUD COVERAGE (%)'},
    {'name': 'sun_sunrise', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'TIME OF THE SUNRISE IN FORMAT "HH:MM:SS"'},
    {'name': 'sun_sunset', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'TIME OF THE SUNSET IN FORMAT "HH:MM:SS"'},
]

In [16]:
## FORECAST 15 DAYS V2
response = requests.request('GET', forecast_15d_v2.format(city = '3477', token = LOGIN['token']))

# Converting into a dataframe
data = json.loads(response.text)
data = pd.json_normalize(data, record_path = ['data'], meta = ['id','name', 'state', 'country'])
data = data[['date', 'datebr', 'id', 'name', 'state', 'country', 'humidity.min', 'humidity.max', 'rain.precipitation', 
             'wind.velocity', 'wind.direction', 'wind.direction_degrees', 'thermal_sensation.min', 'thermal_sensation.max', 
             'temperature.min', 'temperature.max']]
data = data.rename(columns = {
    'date': 'date',
    'datebr': 'date_br',
    'id': 'id',
    'name': 'name',
    'state': 'state',
    'country': 'country',
    'humidity.min': 'humidity_min',
    'humidity.max': 'humidity_max',
    'rain.precipitation': 'rain_precipitation',
    'wind.velocity': 'wind_velocity',
    'wind.direction': 'wind_direction',
    'wind.direction_degrees': 'wind_direction_degrees',
    'thermal_sensation.min': 'thermal_sensation_min',
    'thermal_sensation.max': 'thermal_sensation_max',
    'temperature.min': 'temperature_min',
    'temperature.max': 'temperature_max'
})
data

,date,date_br,id,name,state,country,humidity_min,humidity_max,rain_precipitation,wind_velocity,wind_direction,wind_direction_degrees,thermal_sensation_min,thermal_sensation_max,temperature_min,temperature_max
0,2022-05-18,18/05/2022,3477,São Paulo,SP,BR,40,45,0,8,W,289,19,24,6,15
1,2022-05-19,19/05/2022,3477,São Paulo,SP,BR,46,75,0,6,WSW,250,20,25,7,17
2,2022-05-20,20/05/2022,3477,São Paulo,SP,BR,47,72,0,3,SSW,205,21,26,8,18
3,2022-05-21,21/05/2022,3477,São Paulo,SP,BR,50,90,3,3,SE,153,21,25,10,19
4,2022-05-22,22/05/2022,3477,São Paulo,SP,BR,44,85,0,3,E,109,22,25,10,21
5,2022-05-23,23/05/2022,3477,São Paulo,SP,BR,32,79,0,3,NE,59,22,26,13,23
6,2022-05-24,24/05/2022,3477,São Paulo,SP,BR,37,72,0,3,ENE,87,22,26,14,23


In [17]:
# GBQ schema information
schema_main = [
    {'name': 'date', 'type': 'DATETIME', 'mode': 'NULLABLE', 'description': 'DATE IN FORMAT "YYYY-MM-DD" (EX: 2021-10-01)'},
    {'name': 'date_br', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DATE IN BRAZILIAN FORMAT "DD/MM/YYYY" (EX: 01/10/2021)'},
    {'name': 'id', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY ID'},
    {'name': 'name', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY NAME'},
    {'name': 'state', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY STATE'},
    {'name': 'country', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'CITY COUNTRY'},
    {'name': 'humidity_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MIN (%)'},
    {'name': 'humidity_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'RELATIVE HUMIDITY MAX (%)'},
    {'name': 'rain_precipitation', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'PRECIPITATION IN MILIMETER'},
    {'name': 'wind_velocity', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND INTENSITY IN KM/H'},
    {'name': 'wind_direction', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION IN DEGREES'},
    {'name': 'wind_direction_degrees', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'WIND DIRECTION'},
    {'name': 'thermal_sensation_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM THERMAL SENSATION(°C)'},
    {'name': 'thermal_sensation_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM THERMAL SENSATION(°C)'},
    {'name': 'temperature_min', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MINIMUM TEMPERATURE IN CELSIUS DEGREE'},
    {'name': 'temperature_max', 'type': 'FLOAT', 'mode': 'NULLABLE', 'description': 'MAXIMUM TEMPERATURE IN CELSIUS DERGREE'},
]

In [18]:
## FORECAST 3 DAYS FOR REGION
# Setting empty dataframe outside loop
df = pd.DataFrame()

for region in ['sul','sudeste','norte','nordeste','centro-oeste']:
    
    # Get the response
    response = requests.request('GET', forecast_3d_region.format(region = region, token = LOGIN['token']))

    # Converting into a dataframe
    data = json.loads(response.text)
    data = pd.json_normalize(data, record_path = ['data'], meta = ['region', 'acronym'])
    data = data[['date', 'date_br', 'region', 'acronym', 'text', 'image']]
    
    # Appending the result of the multiples REGIONS
    df = pd.concat([df, data])

df

,date,date_br,region,acronym,text,image
0,2022-05-18,18/05/2022,Sul,SU,"O dia começa com rajadas de ventos bem intensas no RS e em SC, atingindo mais de 70 km/h, mas co...",https://imagens.climatempo.com.br/mapas/yysuprev1.gif
1,2022-05-19,19/05/2022,Sul,SU,"Venta bastante ao longo do dia no leste do RS, SC e do PR. O dia fica com mais nebulosidade no c...",https://imagens.climatempo.com.br/mapas/yysuprev2.gif
2,2022-05-20,20/05/2022,Sul,SU,O tempo firme atua por toda a Região Sul nesta sexta-feira. O sol aparece com variação de nuvens...,https://imagens.climatempo.com.br/mapas/yysuprev3.gif
0,2022-05-18,18/05/2022,Sudeste,SE,O frio aumenta no Sudeste e tem previsão de geada em várias áreas do estado de SP e sul de MG. C...,https://imagens.climatempo.com.br/mapas/yyseprev1.gif
1,2022-05-19,19/05/2022,Sudeste,SE,Amanhecer frio e com geada em SP e sul de MG. Possibilidade de pouca chuva apenas no norte de Mi...,https://imagens.climatempo.com.br/mapas/yyseprev2.gif
2,2022-05-20,20/05/2022,Sudeste,SE,"O tempo fica firme no ES, maior parte de MG, norte e interior do RJ, interior e leste de SP. Reg...",https://imagens.climatempo.com.br/mapas/yyseprev3.gif
0,2022-05-18,18/05/2022,Norte,NO,"Chuva intensa e volumosa segue ocorrendo em quase todos os estados, sendo de forte intensidade n...",https://imagens.climatempo.com.br/mapas/yynoprev1.gif
1,2022-05-19,19/05/2022,Norte,NO,"Céu nublado e chuva frequente em RR, o risco de temporais aumenta por toda a costa norte da Regi...",https://imagens.climatempo.com.br/mapas/yynoprev2.gif
2,2022-05-20,20/05/2022,Norte,NO,"O tempo ainda é destaque no AC, norte de RO e maior parte do AM. Chove forte no norte do PA e to...",https://imagens.climatempo.com.br/mapas/yynoprev3.gif
0,2022-05-18,18/05/2022,Nordeste,NE,"Chuva com trovoadas na BA, PI, RN e oeste de PE. Tempo nublado com variações de abertura de sol ...",https://imagens.climatempo.com.br/mapas/yyneprev1.gif


In [19]:
# GBQ schema information
schema_main = [
    {'name': 'date', 'type': 'DATETIME', 'mode': 'NULLABLE', 'description': 'DATE FORMAT "YYYY-MM-DD" (EX.: 2019-08-18).'},
    {'name': 'date_br', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'DATE IN BRAZILIAN FORMAT "DD/MM/YYYY" (EX.: 18/08/2019).'},
    {'name': 'region', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'REGION COUNTRY.'},
    {'name': 'acronym', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'REGION ACRONYM.'},
    {'name': 'text', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'TEXT WITH DESCRIPTION FOR FORECAST OF DAY.'},
    {'name': 'image', 'type': 'STRING', 'mode': 'NULLABLE', 'description': 'IMAGE URL WITH DAY FORECAST OF REGION'},
]